# **⚖️ EMPLOYMENT LAW: CASE LAW DISCOVERY SYATEM UNDER KENYA LAW**

## **PROJECT OBJECTIVE**

* ### To build a smart legal research tool that helps advocates quickly find accurate, relevant, and authoritative employment law cases from Kenya Law based on legal issues described in plain English.

## **SCOPE**

### Area:

* ### Employment & Labour Relations

### Courts:

* ### Employment and Labour Relations Court (ELRC)

* ### Court of Appeal (employment cases)

* ### Supreme Court (where applicable)

### Jurisdiction:

* ### Kenya

### Output:

* ### Research assistance only (NOT legal advice)

## **METHODOLOGY**

STEP 1: DATA COLLECTION (KENYA LAW)

STEP 2: LEGAL TEXT PREPROCESSING

STEP 3: EMBEDDINGS (CORE INTELLIGENCE)

STEP 4: SEMANTIC SEARCH (THE MAGIC)

STEP 5: RELEVANCE & AUTHORITY RANKING

STEP 6: DISPLAY MEANINGFUL RESULTS

STEP 7: USER INTERFACE (STREAMLIT)

STEP 8: LEGAL & ETHICAL SAFEGUARDS

# **DATA COLLECTION FROM KENYA LAW**

## STEP 1: MANUALLY COLLECT JUDGMENT LINKS

In [1]:
# Importing libraries

import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
import re
import json

import torch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

C:\Users\BEST\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_url = "https://new.kenyalaw.org/judgments/KEELRC/"

In [3]:
headers = {
    "User-Agent": "EmploymentLawResearchBot/1.0 (Academic Project)"
}

response = requests.get(base_url, headers=headers, timeout=20)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

## STEP 2: EXTRACT CORE METADATA

In [4]:
# Case title
title = soup.find("h1")
case_title = title.get_text(strip=True) if title else "Unknown Title"

# Court
court_tag = soup.find("meta", {"name": "court"})
court = court_tag["content"] if court_tag else "ELRC"

# Date
date_tag = soup.find("meta", {"name": "date"})
decision_date = date_tag["content"] if date_tag else "Unknown Date"

print(case_title, court, decision_date)

Employment and Labour Relations Court ELRC Unknown Date


## STEP 3: EXTRACT THE JUDGMENT BODY

In [5]:
for tag in soup.find_all(["article", "section", "div"]):
    text = tag.get_text(strip=True)
    if len(text) > 1000:
        print(tag.name, tag.get("class"))


div ['container', 'pb-5']
div ['row']
div ['col-lg-3', 'd-none', 'd-lg-block', 'document-list-facets-wrapper', 'pocketlaw-hidden']
div ['col']
div ['position-relative']
div ['table-responsive']


In [6]:
judgment_div = soup.find(
    lambda tag: tag.name in ["div", "article", "section"]
    and tag.get_text(strip=True)
    and len(tag.get_text(strip=True)) > 2000
)

if not judgment_div:
    raise ValueError("Judgment body not found")

judgment_text = judgment_div.get_text(separator="\n", strip=True)


In [7]:
print("===== JUDGMENT PREVIEW =====")
print(judgment_text[:700])
print("\n===== END PREVIEW =====")


===== JUDGMENT PREVIEW =====
Employment and Labour Relations Court
31,114
  
    
      judgments
Advanced search
Court stations
Employment and Labour Relations at Garissa
Employment and Labour Relations Court at Bungoma
Employment and Labour Relations Court at Eldoret
Employment and Labour Relations Court at Kakamega
Employment and Labour Relations Court at Kericho
Employment and Labour Relations Court at Kisii
Employment and Labour Relations Court at Kisumu
Employment and Labour Relations Court at Kitale
Employment and Labour Relations Court at Machakos
Employment and Labour Relations Court at Malindi
Employment and Labour Relations Court at Meru
Employment and Labour Relations Court at Mombasa
Employment and Labour R

===== END PREVIEW =====


# **CLEAN LEGAL TEXT TO IMPROVE QUALITY**

In [8]:
def clean_akn_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

clean_text = clean_akn_text(judgment_text)

# **SAVE STRUCTURED OUTPUT**

In [9]:
os.makedirs("data/akn_cases", exist_ok=True)

case_data = {
    "case_title": case_title,
    "court": court,
    "decision_date": decision_date,
    "source_url": base_url,
    "text": clean_text
}

file_name = "keelrc_2025_962.json"

with open(f"data/akn_cases/{file_name}", "w", encoding="utf-8") as f:
    json.dump(case_data, f, ensure_ascii=False, indent=2)

# **SPLIT INTO SEARCHABLE CHUNKS (FOR NLP)**

In [10]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

chunks = chunk_text(clean_text)

# **CREATE EMBEDDINGS**

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)


In [12]:
print(embeddings.shape)

(4, 384)


# **CREATE CHROMADB CLIENT - PERSISTENT**

In [13]:
import typing_extensions

In [14]:
import chromadb
from chromadb.config import Settings

In [15]:
client = chromadb.Client(
    Settings(
        persist_directory="chroma_db",
        anonymized_telemetry=False
    )
)

In [16]:
collection = client.get_or_create_collection(
    name="judgments"
)


In [17]:
# Unique IDs for each chunk
ids = [f"chunk_{i}" for i in range(len(chunks))]

# metadata for each chunk 
metadatas = [
    {
        "case_title": case_title,  # from your scraping
        "court": court,
        "date": decision_date,
        "chunk_index": i
    }
    for i in range(len(chunks))
]

# Add to Chroma
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),  # convert numpy array to list
    ids=ids,
    metadatas=metadatas
)

In [18]:
ids = [f"chunk_{i}" for i in range(len(chunks))]

collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),  # convert numpy → list
    ids=ids,
    metadatas=[{"source": "judgment"} for _ in chunks]
)

In [19]:
print(client.list_collections())

[Collection(name=judgments)]


In [21]:
query = "unfair termination of employment"
query_embedding = model.encode([query])

results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=3
)

In [22]:
# Show top results
for doc in results["documents"][0]:
    print(doc[:300])  # first 300 characters
    print("-" * 40)

Employment and Labour Relations Court 31,114 judgments Advanced search Court stations Employment and Labour Relations at Garissa Employment and Labour Relations Court at Bungoma Employment and Labour Relations Court at Eldoret Employment and Labour Relations Court at Kakamega Employment and Labour R
----------------------------------------
of 2018 (Consolidated)) [2025] KEELRC 3648 (KLR) (17 December 2025) (Ruling) 17 December 2025 Kenya Union of Clinical Officers v Baringo County Public Service Board & 46 others; Public Service Commission & 2 others (Interested Parties) (Judicial Review E027 & E057 of 2025 (Consolidated)) [2025] KEEL
----------------------------------------
Munga (Appeal E009 of 2025) [2025] KEELRC 3607 (KLR) (16 December 2025) (Ruling) 16 December 2025 Juma v Red Mamba Agencies Limited & another (Miscellaneous Application E135 of 2025) [2025] KEELRC 3568 (KLR) (16 December 2025) (Ruling) 16 December 2025 Kamore v Smep Michrofinance Bank Plc (Cause E99
---------------

In [23]:
query = "unfair termination of employment"
query_embedding = model.encode([query])

results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=3
)
print(results)

{'ids': [['chunk_0', 'chunk_2', 'chunk_3']], 'embeddings': None, 'documents': [['Employment and Labour Relations Court 31,114 judgments Advanced search Court stations Employment and Labour Relations at Garissa Employment and Labour Relations Court at Bungoma Employment and Labour Relations Court at Eldoret Employment and Labour Relations Court at Kakamega Employment and Labour Relations Court at Kericho Employment and Labour Relations Court at Kisii Employment and Labour Relations Court at Kisumu Employment and Labour Relations Court at Kitale Employment and Labour Relations Court at Machakos Employment and Labour Relations Court at Malindi Employment and Labour Relations Court at Meru Employment and Labour Relations Court at Mombasa Employment and Labour Relations Court at Nairobi Employment and Labour Relations Court at Nakuru Employment and Labour Relations Court at Nyeri All years 2025 2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2007 2006 2005 2004 2003 20